In [1]:
# coding='utf-8'
import os
import sys
import numpy as np
import time
import datetime
import json
import importlib
import logging
import shutil

import torch
import torch.nn as nn

from yolo_model import yoloModel
from PASCAL_Dataloader import create_split_loaders
from YOLO_Loss import YoloLoss
#from utils import NMS
from bbox import bbox_iou, non_max_suppression

In [2]:
def evaluate(config):
    # Load and initialize network
    net = yoloModel(config)

    # Set data parallel
    net = net.cuda()

    # Restore pretrain model
    if config["pretrain_snapshot"]:
        state_dict = torch.load(config["pretrain_snapshot"])
        net.load_state_dict(state_dict)
    else:
        logging.warning("missing pretrain_snapshot!!!")

    # YOLO loss with 3 scales
    yolo_losses = []
    for i in range(3):
        yolo_losses.append(YoloLoss(config["classes"], (config["img_w"], config["img_h"]), config["anchors"][i]))

    # DataLoader
    root_dir = os.getcwd()
    train_loader, val_loader, test_loader = create_split_loaders(root_dir, config['batch_size'])

    # Start the eval loop
    logging.info("Start eval.")
    n_gt = 0
    correct = 0
    for step, samples in enumerate(test_loader):
        images, labels = samples["image"], samples["label"]
        images,labels = images.cuda(),labels.cuda()
        with torch.no_grad():
            outputs = net(images)
            output_list = []
            for i in range(3):
                output_list.append(yolo_losses[i](outputs[i]))
            output = torch.cat(output_list, 1)
            output = non_max_suppression(output, config["classes"], conf_thres=0.5)
            #  calculate
            for sample_i in range(labels.size(0)):
                # Get labels for sample where width is not zero (dummies)
                target_sample = labels[sample_i, labels[sample_i, :, 3] != 0]
                for obj_cls, tx, ty, tw, th in target_sample:
                    # Get rescaled gt coordinates
                    tx1, tx2 = config["img_w"] * (tx - tw / 2), config["img_w"] * (tx + tw / 2)
                    ty1, ty2 = config["img_h"] * (ty - th / 2), config["img_h"] * (ty + th / 2)
                    n_gt += 1
                    box_gt = torch.cat([coord.unsqueeze(0) for coord in [tx1, ty1, tx2, ty2]]).view(1, -1)
                    box_gt = box_gt.float()
                    sample_pred = output[sample_i]
                    #print(sample_pred)
                    #print(obj_cls, tx1,ty1,tx2,ty2)
                    if sample_pred is not None:
                        # Iterate through predictions where the class predicted is same as gt
                        for x1, y1, x2, y2, conf, obj_conf, obj_pred in sample_pred[sample_pred[:, 6] == obj_cls.float()]:
                            box_pred = torch.cat([coord.unsqueeze(0) for coord in [x1, y1, x2, y2]]).view(1, -1)
                            print('Pred:',box_pred)
                            print('GT:',box_gt)
                            iou = bbox_iou(box_pred, box_gt)
                            if iou >= config["confidence_threshold"]:
                                correct += 1
                                break
        if n_gt:
            logging.info('Batch [%d/%d] mAP: %.5f' % (step, len(test_loader), float(correct / n_gt)))

    logging.info('Mean Average Precision: %.5f' % float(correct / n_gt))

In [3]:
def main():
    logging.basicConfig(level=logging.DEBUG,
                        format="[%(asctime)s %(filename)s] %(message)s")

    config = {}
    config["batch_size"] = 1
    config['backbone_name'] = "darknet_53"
    config['backbone_pretrained'] = ""
    config['anchors'] = [[[116, 90], [156, 198], [373, 326]],
                                [[30, 61], [62, 45], [59, 119]],
                                [[10, 13], [16, 30], [33, 23]]]
    config['classes'] = 20
    config['img_h'] = config['img_w'] = 416
    config['confidence_threshold'] = 0.5
    config['pretrain_snapshot'] = "./states/20190530222509/model.pth"
    config['classes_names_path'] = "./data/voc.names"
    config['images_path'] = "./images"

    # Start training
    evaluate(config)

if __name__ == "__main__":
    main()

[2019-05-31 10:46:28,688 <ipython-input-2-7667fb18f09c>] Start eval.
[2019-05-31 10:46:31,237 <ipython-input-2-7667fb18f09c>] Batch [0/2077] mAP: 0.16667
[2019-05-31 10:46:31,318 <ipython-input-2-7667fb18f09c>] Batch [1/2077] mAP: 0.28571
[2019-05-31 10:46:31,350 <ipython-input-2-7667fb18f09c>] Batch [2/2077] mAP: 0.22222


Pred: tensor([[153.5014, 221.7497, 189.1909, 310.3811]], device='cuda:0')
GT: tensor([[163.6098, 244.6080, 195.3171, 297.8560]], device='cuda:0')
Pred: tensor([[257.0607, 229.7418, 293.9885, 298.7388]], device='cuda:0')
GT: tensor([[163.6098, 244.6080, 195.3171, 297.8560]], device='cuda:0')
Pred: tensor([[153.5014, 221.7497, 189.1909, 310.3811]], device='cuda:0')
GT: tensor([[221.9512, 235.4560, 260.0000, 295.3600]], device='cuda:0')
Pred: tensor([[257.0607, 229.7418, 293.9885, 298.7388]], device='cuda:0')
GT: tensor([[221.9512, 235.4560, 260.0000, 295.3600]], device='cuda:0')
Pred: tensor([[153.5014, 221.7497, 189.1909, 310.3811]], device='cuda:0')
GT: tensor([[285.3658, 237.1200, 323.4146, 294.5280]], device='cuda:0')
Pred: tensor([[257.0607, 229.7418, 293.9885, 298.7388]], device='cuda:0')
GT: tensor([[285.3658, 237.1200, 323.4146, 294.5280]], device='cuda:0')
Pred: tensor([[153.5014, 221.7497, 189.1909, 310.3811]], device='cuda:0')
GT: tensor([[139.5122, 240.4480, 169.9512, 298.688

[2019-05-31 10:46:31,391 <ipython-input-2-7667fb18f09c>] Batch [3/2077] mAP: 0.27273
[2019-05-31 10:46:31,437 <ipython-input-2-7667fb18f09c>] Batch [4/2077] mAP: 0.38462
[2019-05-31 10:46:31,475 <ipython-input-2-7667fb18f09c>] Batch [5/2077] mAP: 0.42857
[2019-05-31 10:46:31,512 <ipython-input-2-7667fb18f09c>] Batch [6/2077] mAP: 0.46667
[2019-05-31 10:46:31,543 <ipython-input-2-7667fb18f09c>] Batch [7/2077] mAP: 0.38889
[2019-05-31 10:46:31,574 <ipython-input-2-7667fb18f09c>] Batch [8/2077] mAP: 0.36842


Pred: tensor([[161.3467, 144.7428, 270.9502, 396.1806]], device='cuda:0')
GT: tensor([[146.4320, 147.5413, 259.5840, 392.7040]], device='cuda:0')
Pred: tensor([[161.3467, 144.7428, 270.9502, 396.1806]], device='cuda:0')
GT: tensor([[199.6800, 166.4000, 301.1840, 388.2667]], device='cuda:0')
Pred: tensor([[206.1532, 218.3599, 265.8383, 312.7794]], device='cuda:0')
GT: tensor([[162.2400, 210.4762, 207.1680, 321.9048]], device='cuda:0')
Pred: tensor([[150.9110, 192.5718, 210.3128, 336.0300]], device='cuda:0')
GT: tensor([[162.2400, 210.4762, 207.1680, 321.9048]], device='cuda:0')
Pred: tensor([[206.1532, 218.3599, 265.8383, 312.7794]], device='cuda:0')
GT: tensor([[213.8240, 210.4762, 274.5600, 323.1429]], device='cuda:0')
Pred: tensor([[191.2173, 197.3741, 331.7399, 298.4532]], device='cuda:0')
GT: tensor([[120.6400, 188.2190, 357.7600, 299.7118]], device='cuda:0')
Pred: tensor([[101.1174,  70.2710, 323.1919, 295.2101]], device='cuda:0')
GT: tensor([[ 58.8421,  31.4502, 350.3158, 331.656

[2019-05-31 10:46:31,611 <ipython-input-2-7667fb18f09c>] Batch [9/2077] mAP: 0.31818
[2019-05-31 10:46:31,652 <ipython-input-2-7667fb18f09c>] Batch [10/2077] mAP: 0.32000
[2019-05-31 10:46:31,710 <ipython-input-2-7667fb18f09c>] Batch [11/2077] mAP: 0.30769
[2019-05-31 10:46:31,751 <ipython-input-2-7667fb18f09c>] Batch [12/2077] mAP: 0.28571
[2019-05-31 10:46:31,789 <ipython-input-2-7667fb18f09c>] Batch [13/2077] mAP: 0.25000
[2019-05-31 10:46:31,826 <ipython-input-2-7667fb18f09c>] Batch [14/2077] mAP: 0.23529


...

[2019-05-31 10:48:19,974 <ipython-input-2-7667fb18f09c>] Batch [2068/2077] mAP: 0.24192
[2019-05-31 10:48:20,015 <ipython-input-2-7667fb18f09c>] Batch [2069/2077] mAP: 0.24183
[2019-05-31 10:48:20,070 <ipython-input-2-7667fb18f09c>] Batch [2070/2077] mAP: 0.24174
[2019-05-31 10:48:20,119 <ipython-input-2-7667fb18f09c>] Batch [2071/2077] mAP: 0.24188
[2019-05-31 10:48:20,179 <ipython-input-2-7667fb18f09c>] Batch [2072/2077] mAP: 0.24188


Pred: tensor([[332.8380, 146.2014, 354.5612, 207.5418]], device='cuda:0')
GT: tensor([[339.4560, 168.6187, 363.5840, 208.5547]], device='cuda:0')
Pred: tensor([[101.5426,  62.2999, 262.2227, 360.2790]], device='cuda:0')
GT: tensor([[ 94.8480,  25.1789, 368.5760, 410.5263]], device='cuda:0')
Pred: tensor([[ 32.4232,  48.2249, 436.8276, 375.8412]], device='cuda:0')
GT: tensor([[  0.0000,   0.0000, 415.1680, 414.9600]], device='cuda:0')
Pred: tensor([[ 65.7604, 142.8221, 234.7059, 381.4711]], device='cuda:0')
GT: tensor([[ 57.4080, 116.4800, 274.5600, 414.8907]], device='cuda:0')


[2019-05-31 10:48:20,235 <ipython-input-2-7667fb18f09c>] Batch [2073/2077] mAP: 0.24175
[2019-05-31 10:48:20,297 <ipython-input-2-7667fb18f09c>] Batch [2074/2077] mAP: 0.24180
[2019-05-31 10:48:20,430 <ipython-input-2-7667fb18f09c>] Batch [2075/2077] mAP: 0.24150
[2019-05-31 10:48:20,475 <ipython-input-2-7667fb18f09c>] Batch [2076/2077] mAP: 0.24178


Pred: tensor([[ 30.8931, 148.9504, 142.7956, 402.0780]], device='cuda:0')
GT: tensor([[ 36.6080, 165.9391, 135.6160, 414.8477]], device='cuda:0')
Pred: tensor([[ 30.8931, 148.9504, 142.7956, 402.0780]], device='cuda:0')
GT: tensor([[ 71.5520,  44.9418, 153.9200, 239.6898]], device='cuda:0')
Pred: tensor([[ 30.8931, 148.9504, 142.7956, 402.0780]], device='cuda:0')
GT: tensor([[256.2560, 142.8920, 381.0560, 414.8477]], device='cuda:0')
Pred: tensor([[ 59.4697, 132.9512, 281.4747, 395.5777]], device='cuda:0')
GT: tensor([[ 58.2400,  86.1982, 247.1040, 414.7508]], device='cuda:0')
Pred: tensor([[ 59.4697, 132.9512, 281.4747, 395.5777]], device='cuda:0')
GT: tensor([[326.1440,  73.7057, 353.6000, 187.3874]], device='cuda:0')
Pred: tensor([[253.6291,  95.2695, 289.3322, 177.1430]], device='cuda:0')
GT: tensor([[326.1440,  73.7057, 353.6000, 187.3874]], device='cuda:0')
Pred: tensor([[ 59.4697, 132.9512, 281.4747, 395.5777]], device='cuda:0')
GT: tensor([[282.8800,  99.9399, 297.8560, 163.651

[2019-05-31 10:48:21,316 <ipython-input-2-7667fb18f09c>] Mean Average Precision: 0.24178
